In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [15]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]

### 격자(매핑용)

In [13]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [14]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 시간대별 유동인구

In [9]:
# 원도심 영역(도시재생 대상지역)
floating_population_df_time = pd.read_csv('SBJ_2309_001/3.청주시_시간대별_유동인구.csv')
floating_population_df_time['STD_YM'] = floating_population_df_time['STD_YM'].astype(str)
floating_population_df_time.rename(columns={'STD_YM':'연도'}, inplace=True)
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
floating_population_df_time = geo_transform(floating_population_df_time)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
floating_population_df_time = floating_population_df_time[floating_population_df_time['geometry'].within(polygon)]

# 열 이름 변경 과정
columns_to_rename = floating_population_df_time.columns[1:-3] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    numeric_part = column_name.split('_')[-1] # 숫자 부분 추출
    new_column_name = f"{numeric_part}시" # '시'를 붙여 새로운 열 이름 생성
    new_column_names.append(new_column_name) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_time.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
floating_population_df_time

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,연도,00시,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시,lon,lat,geometry
82848,202001,1.17,1.09,1.02,0.97,0.99,1.04,1.29,1.75,2.44,2.65,2.81,3.04,3.09,3.30,3.44,3.46,3.32,3.21,2.86,2.14,1.84,1.66,1.41,1.18,127.483602,36.631043,POINT (127.48360 36.63104)
82849,202001,2.90,2.86,2.75,2.67,2.67,2.79,3.23,4.15,5.47,5.79,6.00,6.55,6.57,7.01,7.31,7.45,7.36,7.17,6.87,5.23,4.45,4.01,3.50,2.98,127.483602,36.631494,POINT (127.48360 36.63149)
82850,202001,2.88,2.81,2.70,2.63,2.61,2.75,3.23,4.29,5.97,6.16,6.23,6.67,6.73,7.19,7.42,7.52,7.38,7.26,6.97,5.30,4.52,4.08,3.57,3.00,127.483602,36.631944,POINT (127.48360 36.63194)
82851,202001,1.47,1.32,1.15,1.04,1.04,1.18,1.70,2.84,4.63,4.50,4.63,5.05,5.23,5.49,5.72,5.56,5.70,5.84,5.86,4.04,3.13,2.84,2.22,1.59,127.483602,36.632395,POINT (127.48360 36.63240)
83145,202001,1.13,1.13,0.99,0.90,0.90,0.97,1.38,2.05,3.39,3.37,3.36,3.53,3.57,3.87,4.03,3.99,3.88,3.94,3.76,2.81,2.30,2.01,1.62,1.31,127.484162,36.630592,POINT (127.48416 36.63059)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983417,202212,0.67,0.62,0.35,0.32,0.30,0.48,1.24,2.44,4.56,4.13,3.96,4.17,4.45,4.91,4.91,4.68,4.82,5.05,5.10,3.25,2.42,2.01,1.54,0.95,127.492551,36.631945,POINT (127.49255 36.63195)
3983418,202212,3.23,2.68,1.22,1.32,1.04,1.57,3.66,9.46,17.00,15.22,15.52,16.48,17.34,19.28,19.48,19.90,21.10,22.53,22.34,15.15,12.01,10.67,7.80,4.89,127.492550,36.632396,POINT (127.49255 36.63240)
3983419,202212,1.02,1.70,0.79,0.62,0.58,0.99,1.80,3.30,6.15,5.35,5.60,5.67,5.67,6.66,6.57,6.22,6.71,6.78,6.66,4.87,3.73,3.09,2.37,1.70,127.492550,36.632847,POINT (127.49255 36.63285)
3983801,202212,1.84,1.22,1.08,0.92,1.34,2.93,7.50,12.57,16.25,14.74,16.07,19.76,21.12,21.17,19.76,19.11,20.15,21.10,18.63,11.28,7.96,6.39,4.01,2.63,127.493110,36.628339,POINT (127.49311 36.62834)


#### 유동인구 HEATMAP의 시계열 변화 시각화

In [46]:
# 딕셔너리 생성 및 연도 & DataFrame 매핑
filtered_floating_population_time = {}
year_list = ['2020', '2021', '2022']

for year in year_list:
    year_df = pd.DataFrame({'연도' : [year]}) # 해당 연도 df 설정(접합용)
    year_tot = pd.DataFrame() # 해당 연도 df 설정(최종용)
    floating_population_df_time_year = floating_population_df_time[floating_population_df_time['연도'].str.startswith(year)] # 특정 연도로만 필터링
    for point in floating_population_df_time_year['geometry'].unique():
        point_df = pd.DataFrame({'lon' : [point.x], 'lat' : [point.y]}) # 해당 지점 df 설정(접합용)
        fpt_point_df = floating_population_df_time_year[floating_population_df_time_year['geometry'] == point]
        
        # 1월부터 12월 까지의 월별 유동인구 평균계산 및 연도, 지점 df와 접합 & 최종 df에 접합
        column_mean = fpt_point_df.iloc[:,1:25].mean()
        column_mean_df = pd.DataFrame(column_mean).transpose()
        column_mean_df = column_mean_df.iloc[:, :24].round(2) # 소수점 2번째 자리까지만 표기
        column_mean_df = pd.concat([year_df, column_mean_df, point_df], axis=1)
        year_tot = pd.concat([year_tot, column_mean_df])
    else:
        # 유동인구 정규화 과정
        max_val = year_tot.iloc[:, 1:-2].values.max() # 유동인구의 최댓값
        year_tot.iloc[:, 1:-2] = (year_tot.iloc[:, 1:-2] / max_val) * 100 # 최댓값으로 나누어 100을 곱하여 정규화
        filtered_floating_population_time[year] = year_tot

#### 2020년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2020']['lat']
longitudes = filtered_floating_population_time['2020']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2020'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2021년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2021']['lat']
longitudes = filtered_floating_population_time['2021']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2021'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2022년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_time['2022']['lat']
longitudes = filtered_floating_population_time['2022']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for i in range(24):
    time_column = f'{i:02}시'
    volume = filtered_floating_population_time['2022'][time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 격자에 매핑 & 모든 시간대를 통합한 격자 유동인구 시각화

In [61]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_tottime = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = floating_population_df_time[floating_population_df_time['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in floating_population_df_time['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:25].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_tottime에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:25].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        column_means_2020_df = pd.DataFrame(column_means_2020_df.sum(axis=1), columns=['유동인구'])
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:25].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        column_means_2021_df = pd.DataFrame(column_means_2021_df.sum(axis=1), columns=['유동인구'])
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:25].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        column_means_2022_df = pd.DataFrame(column_means_2022_df.sum(axis=1), columns=['유동인구'])
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_tottime = pd.concat([filtered_floating_population_tottime, tot_df])

filtered_floating_population_tottime

100%|██████████| 156/156 [00:27<00:00,  5.78it/s]


,연도,유동인구,geometry
0,2020,111.131667,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2021,86.598333,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2022,103.125833,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2020,1666.923333,POLYGON ((127.48881291670402 36.64430329023720...
0,2021,2007.956667,POLYGON ((127.48881291670402 36.64430329023720...
...,...,...,...
0,2021,137.588333,POLYGON ((127.49105137605241 36.63528850233756...
0,2022,183.466667,POLYGON ((127.49105137605241 36.63528850233756...
0,2020,46.596667,POLYGON ((127.48433936092893 36.63799230366292...
0,2021,39.403333,POLYGON ((127.48433936092893 36.63799230366292...


In [ ]:
#  고령화 정도에 따라 grid색상을 지정하는 함수
def rate_color(flopop):
    if flopop > 1091.674375:
        return 'red'
    elif 1091.674375 > flopop >= 399.525000:
        return 'orange'
    elif 399.525000 > flopop >= 103.005833:
        return 'yellow'
    elif 103.005833 > flopop >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2020, 2023)]
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    filtered_floating_population_tottime_year = filtered_floating_population_tottime[filtered_floating_population_tottime['연도'] == year]

    # grid 값을 기준으로 그룹화
    for grid in filtered_floating_population_tottime_year['geometry'].unique():
        subset = filtered_floating_population_tottime_year[filtered_floating_population_tottime_year['geometry'] == grid].copy()
        flopop = subset['유동인구'].iloc[0]
        color = rate_color(flopop)
        subset['유동인구'] = subset['유동인구'].apply(lambda x: str(round(x, 2)) + '%')
        popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset['유동인구'].iloc[0]}</div>"
        folium.GeoJson(
            subset['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m